In [1]:
import os
import time
import requests
from arcgis.gis import GIS
from arcgis.raster import ImageryLayer
import geopandas as gpd
from shapely.geometry import box
import rasterio
from rasterio.mask import mask
from rasterio.merge import merge
from rasterio.io import MemoryFile
 
def create_tiles(extent, tile_size):
    xmin, ymin, xmax, ymax = extent
    xstep, ystep = tile_size
 
    tiles = []
    for x in range(int(xmin), int(xmax), xstep):
        for y in range(int(ymin), int(ymax), ystep):
            tiles.append((x, y, x + xstep, y + ystep))
    return tiles
 
def buffer_geometry(geometry, buffer_distance):
    """
    Create a buffered geometry around the input geometry.
    """
    return geometry.buffer(buffer_distance)
 
def clip_raster_with_geometry(input_raster, geometry, output_raster):
    """
    Clip the raster using the buffered geometry and save the result.
    """
    try:
        with rasterio.open(input_raster) as src:
            # Convert geometry to GeoJSON format
            geojson_geometry = [geometry.__geo_interface__]
            # Clip the raster using the geometry
            out_image, out_transform = mask(src, geojson_geometry, crop=True)
            out_meta = src.meta.copy()
 
            # Update metadata with new dimensions and transform
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform
            })
 
            # Write the clipped raster to a file
            with rasterio.open(output_raster, "w", **out_meta) as dest:
                dest.write(out_image)
        print(f"Clipped raster saved: {output_raster}")
    except Exception as e:
        print(f"Error clipping raster: {e}")
 
def process_tile(secure_img_lyr, portal, xmin, ymin, xmax, ymax, i, output_folder):
    try:
        export_result = secure_img_lyr.export_image(
            bbox=[xmin, ymin, xmax, ymax],
            image_sr=3857,
            size=[1024, 1024],
            export_format='tiff'
        )
        dem_url = export_result['href']
        token = portal._con.token
        dem_url_with_token = f"{dem_url}&token={token}"
        response = requests.get(dem_url_with_token)
        response.raise_for_status()
 
        memfile = MemoryFile(response.content)
        dataset = memfile.open()
        tile_path = os.path.join(output_folder, f"Tile_{i}.tif")
        dataset.meta.update({"driver": "GTiff"})
        with rasterio.open(tile_path, 'w', **dataset.meta) as dest:
            dest.write(dataset.read(1), 1)
        print(f"Tile {i + 1} processed successfully")
        return tile_path
    except requests.exceptions.RequestException as e:
        print(f"HTTP request error for Tile {i}: {e}")
    except Exception as e:
        print(f"Error processing DEM for Tile {i}: {e}")
    return None
 
def run_model_builder_script():
    portal = GIS(url='https://auth-spatial.information.qld.gov.au/arcgis/', username='svc_scs_rdmwrs ', password='f7w1W!RIoyMG', verify_cert=False)
    secure_url = 'https://auth-spatial-img.information.qld.gov.au/arcgis/rest/services/Elevation/DEM_TimeSeries_CLPUsers/ImageServer'
    secure_img_lyr = ImageryLayer(secure_url, portal)
    output_folder = r'C:\projects\New_Script_SCP\outputs2\sample_6_new'
    output_mosaic_path = os.path.join(output_folder, "Merged_DEM6.tif")
    tile_size = (1000, 1000)
    buffer_distance = 50  # In meters
    shapefile_path = r'C:\projects\New_Script_SCP\outputs2\sample_5\sample_5.shp'
    try:
        gdf = gpd.read_file(shapefile_path)
        if gdf.empty:
            print("Error: Shapefile is empty.")
            return
    except Exception as e:
        print(f"Error loading shapefile: {e}")
        return
    gdf = gdf.to_crs(epsg=3857)
    gdf_buffered = gdf.buffer(buffer_distance)
    buffered_extent = gdf_buffered.total_bounds
    tiles = create_tiles(buffered_extent, tile_size)
    print(f"Created {len(tiles)} tiles for the buffered AOI.")
    dem_tiles = []
    start_time = time.time()
    for i, (xmin, ymin, xmax, ymax) in enumerate(tiles):
        print(f"Processing Tile {i + 1}/{len(tiles)} with bounds: {xmin}, {ymin}, {xmax}, {ymax}")
        tile_path = process_tile(secure_img_lyr, portal, xmin, ymin, xmax, ymax, i, output_folder)
        if tile_path:
            # Clip each tile using the buffered geometry
            for idx, buffered_geometry in gdf_buffered.items():  # Corrected iteration
                output_clipped_tile_path = os.path.join(output_folder, f"Clipped_Tile_{i}.tif")
                clip_raster_with_geometry(tile_path, buffered_geometry, output_clipped_tile_path)
                dem_tiles.append(output_clipped_tile_path)
 
    print("Merging DEM tiles....")
    if dem_tiles:
        try:
            src_files_to_mosaic = [rasterio.open(tile) for tile in dem_tiles]
            mosaic, out_transform = merge(src_files_to_mosaic)
            mosaic = mosaic.squeeze()  # Remove any extra dimensions
            with rasterio.open(output_mosaic_path, 'w', driver='GTiff',
                               height=mosaic.shape[0], width=mosaic.shape[1], 
                               count=1, dtype=mosaic.dtype,
                               crs='EPSG:3857', transform=out_transform) as dst:
                dst.write(mosaic, 1)
 
            print(f"Merged DEM saved to {output_mosaic_path}")
        except Exception as e:
            print(f"Error during merging: {e}")
    print("Total execution time:", time.time() - start_time)
 
if __name__ == '__main__':
    run_model_builder_script()

Setting `verify_cert` to False is a security risk, use at your own risk.


Created 2 tiles for the buffered AOI.
Processing Tile 1/2 with bounds: 16528201, -3254774, 16529201, -3253774
Tile 1 processed successfully
Clipped raster saved: C:\projects\New_Script_SCP\outputs2\sample_6_new\Clipped_Tile_0.tif
Processing Tile 2/2 with bounds: 16528201, -3253774, 16529201, -3252774
Tile 2 processed successfully
Clipped raster saved: C:\projects\New_Script_SCP\outputs2\sample_6_new\Clipped_Tile_1.tif
Merging DEM tiles....
Merged DEM saved to C:\projects\New_Script_SCP\outputs2\sample_6_new\Merged_DEM6.tif
Total execution time: 13.934037446975708
